In [116]:
def rev(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return complement[seq]
rev("A")

'T'

In [122]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen")
refseq=Path("/home/davide/Desktop/genomiChro/annotati_Refseq")
for file in dir.glob("**/fimo.tsv"):
    motivi=pd.read_csv(file,sep="\t",header=0,skipfooter=4,engine='python')
    motivi=motivi[["sequence_name",'matched_sequence']]
    motivi.columns=["locus","motivo"]
    motivi.set_index("locus",inplace=True)
import numpy as np
df.index.name="locus"
#transform df in dataframe
df=pd.DataFrame(df)
df=pd.concat([df,motivi],axis=1)
#append motif in motivo column to the list of motifs in motivi column
df["motivo"]=df["motivo"].apply(lambda x: [x])
df["motivi"] = df.apply(lambda row: row["motivi"] + row["motivo"] if row["motivo"] != [np.nan] else row["motivi"], axis=1)
#drop motivo column
df.drop(columns="motivo",inplace=True)
df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [115]:
df.describe()

count                  2396
unique                 2314
top       [cTAGTATGACTCCTc]
freq                      8
Name: motivi, dtype: object

In [136]:
from bs4 import BeautifulSoup
from pprint import pprint
file=("/home/davide/Downloads/KEGGall.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
#delete first elements unitl one has Metabolic patwhays in text
for i, li in enumerate(li_tags):
    if "Metabolic pathways" in str(li):
        break
li_tags=li_tags[i:]

pathways = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    pathway=str(li.contents[1])[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")
            #strip the white spaces from the protein ids and also \n
            pids=[pid.strip() for pid in pids]

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    pathways[pathway]=diz
pathways
#create a dataframe from the dictionary
df_path=pd.DataFrame(pathways)

In [137]:

df_path.to_pickle("/home/davide/Downloads/pathways.pkl")

In [88]:

#select only row CCME_29 and explode for pid
df_path=df_path.loc["CCMEE_29"].explode()
df_path=df_path.reset_index()
#set idnex to pid
df_path=df_path.set_index("CCMEE_29")
df_path=df_path.rename(columns={"index":"pathways"})
df_path.index.name="pid"
df_path=df_path.groupby("pid")["pathways"].apply(lambda x: ", ".join(x))
df_path



pid
                  Biosynthesis of cofactors, Pentose and glucuro...
WP_009632395.1                                 Two-component system
WP_250121154.1    Metabolic pathways, O-Antigen repeat unit bios...
WP_250121155.1                   Metabolic pathways, Photosynthesis
WP_250121157.1                                     ABC transporters
                                        ...                        
WP_275983377.1    Metabolic pathways, Biosynthesis of nucleotide...
WP_275983383.1        Metabolic pathways, Oxidative phosphorylation
WP_275983386.1    Metabolic pathways, Biosynthesis of secondary ...
WP_275983394.1                   Metabolic pathways, Photosynthesis
WP_275983398.1                                       Quorum sensing
Name: pathways, Length: 1346, dtype: object

In [130]:
file=("/home/davide/Downloads/KEGGBrite.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
for i, li in enumerate(li_tags):
    if "Enzymes" in str(li):
        break
li_tags = li_tags[i:]

brites = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    brite=li.contents[0].get_text()+" - "+li.contents[1].get_text()[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(",")

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    brites[brite]=diz
df_br=pd.DataFrame(brites)
df_br


,ko01000 - Enzymes,ko01001 - Protein kinases,ko01009 - Protein phosphatases and associated proteins,ko01002 - Peptidases and inhibitors,ko01003 - Glycosyltransferases,ko01005 - Lipopolysaccharide biosynthesis proteins,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01004 - Lipid biosynthesis proteins,ko01008 - Polyketide biosynthesis proteins,ko01006 - Prenyltransferases,...,ko02022 - Two-component system,ko02035 - Bacterial motility proteins,ko04812 - Cytoskeleton proteins,ko04147 - Exosome,ko02048 - Prokaryotic defense system,ko04030 - G protein-coupled receptors,ko01504 - Antimicrobial resistance genes,ko00536 - Glycosaminoglycan binding proteins,ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,ko03200 - Viral proteins
SAG_2025,"[WP_317105300.1, WP_317107603.1, WP_317111516....","[, WP_317107242.1, WP_317111031.1, WP_31710546...","[WP_317108776.1, WP_317109651.1]","[WP_317105484.1, WP_317107752.1, WP_317105950....","[WP_317107296.1, WP_317107311.1, WP_317108247....","[WP_317106388.1, WP_317105852.1, WP_317106389....","[WP_317105101.1, WP_317110720.1, WP_317107964....","[WP_317104897.1, WP_317107573.1, WP_317108586....","[WP_317107034.1, WP_317107376.1, WP_317110635....","[WP_317104870.1, WP_317105768.1, WP_317109414....",...,"[, WP_039710300.1, WP_317107241.1, WP_31711098...","[WP_317104584.1, WP_317107749.1, WP_317109829....","[WP_039716527.1, WP_039718890.1, WP_317106402....","[WP_317107603.1, WP_317111516.1, WP_317110195....","[WP_317104733.1, WP_317104968.1, WP_317111784....",[WP_106166095.1],"[WP_317110266.1, WP_317105359.1, WP_317109905....",[WP_317107980.1],"[WP_317107894.1, WP_317108774.1, WP_317107980.1]",[]
PCC_7203,"[WP_015153369.1, WP_015153057.1, WP_015153963....","[, WP_015155183.1, WP_015152962.1, WP_01515482...","[WP_015154644.1, WP_015157009.1, WP_015155893.1]","[WP_015155070.1, WP_015155462.1, WP_015154906....","[WP_015154066.1, WP_199197586.1, WP_015155019....","[WP_015154305.1, WP_015153708.1, WP_015154304....","[WP_015153620.1, WP_281168663.1, WP_015156102....","[WP_015152569.1, WP_015155294.1, WP_041462544....","[WP_015152492.1, WP_015153983.1, WP_015154181....","[WP_219336057.1, WP_015153782.1, WP_015157365....",...,"[, WP_015155182.1, WP_219335965.1, WP_01515518...","[WP_015152121.1, WP_015152715.1, WP_015155460....","[WP_015154286.1, WP_015155181.1, WP_015163027....","[, WP_015152852.1, WP_015152915.1, WP_01515376...","[WP_015152263.1, WP_015152561.1, WP_015157317....",[WP_015156224.1],"[WP_015152889.1, WP_015152965.1, WP_015153420....",[WP_015156087.1],"[WP_015154645.1, WP_015156175.1, WP_015163222....",[]
CCNUC1,"[WP_250016076.1, WP_250017807.1, WP_250015754....","[, WP_250012496.1, WP_250012359.1, WP_25001563...","[WP_015154644.1, WP_250014349.1, WP_250013030.1]","[WP_250012385.1, WP_250012667.1, WP_250012176....","[WP_250016561.1, WP_199197586.1, WP_015155019....","[WP_250016751.1, WP_250016294.1, WP_250016749....","[WP_250016250.1, WP_015156851.1, WP_250013121....","[WP_250012585.1, WP_250013004.1, WP_250013015....","[WP_250012823.1, WP_250015172.1, WP_250016506....","[WP_250015077.1, WP_015157365.1, WP_250016336....",...,"[, WP_015155182.1, WP_250013012.1, WP_25001800...","[WP_015152715.1, WP_250012665.1, WP_250014822....","[WP_015154286.1, WP_250012495.1, WP_250017875....","[WP_250017807.1, WP_015152852.1, WP_015152915....","[WP_250014990.1, WP_250017891.1, WP_051033284....",[WP_015156224.1],"[WP_106166148.1, WP_250015643.1, WP_250016117....",[WP_015156087.1],"[WP_106166068.1, WP_106218645.1, WP_015156087.1]",[WP_250016437.1]
CCMEE_29,"[WP_250122538.1, WP_250124344.1, WP_250124395....","[, WP_250126829.1, WP_250124762.1, WP_25012479...","[WP_250121529.1, WP_250125000.1, WP_250122721.1]","[WP_250121282.1, WP_250121169.1, WP_250121347....","[, WP_250124180.1, WP_250125862.1, WP_25012255...","[WP_250124050.1, WP_250121444.1, WP_250124049....","[WP_250123671.1, WP_250123642.1, WP_250121270....","[WP_250121640.1, WP_250121757.1, WP_250122377....","[, WP_250124307.1, W

In [131]:
#save
df_br.to_pickle("/home/davide/Downloads/brite.pkl")

In [126]:
df_br=df_br.loc["CCMEE_29"].explode()
df_br=df_br.reset_index()
df_br=df_br.set_index("CCMEE_29")
df_br=df_br.rename(columns={"index":"Kegg Brite"})
df_br.index.name="protein_id"
df_br=df_br.groupby("protein_id")["Kegg Brite"].apply(lambda x: ", ".join(x))
df_br

ko01000 - Enzymes                                                 WP_250122538.1
ko01000 - Enzymes                                                 WP_250124344.1
ko01000 - Enzymes                                                 WP_250124395.1
ko01000 - Enzymes                                                 WP_250125460.1
ko01000 - Enzymes                                                 WP_256478781.1
                                                                       ...      
ko01504 - Antimicrobial resistance genes                          WP_250126726.1
ko00536 - Glycosaminoglycan binding proteins                                    
ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins    WP_250121531.1
ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins    WP_250126561.1
ko03200 - Viral proteins                                          WP_250126373.1
Name: CCMEE_29, Length: 2768, dtype: object

CCMEE_29
                  ko01001 - Protein kinases, ko01003 - Glycosylt...
WP_041548398.1                        ko03012 - Translation factors
WP_250121154.1    ko01000 - Enzymes, ko01003 - Glycosyltransfera...
WP_250121155.1    ko01000 - Enzymes, ko00194 - Photosynthesis pr...
WP_250121157.1            ko01000 - Enzymes, ko02000 - Transporters
                                        ...                        
WP_275983388.1    ko03110 - Chaperones and folding catalysts, ko...
WP_275983394.1                    ko00194 - Photosynthesis proteins
WP_275983395.1                               ko02000 - Transporters
WP_275983396.1                 ko02048 - Prokaryotic defense system
WP_275983398.1    ko01002 - Peptidases and inhibitors, ko03110 -...
Name: index, Length: 2011, dtype: object

In [91]:
#find all multiple protein_id in genbank file
from Bio import SeqIO
file="/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
diz={}
lista=[]
for record in SeqIO.parse(file, "genbank"):
    for feature in record.features:
        if feature.type=="CDS":
            locus_tag=feature.qualifiers["locus_tag"][0]
            if "protein_id" in feature.qualifiers:
                product=feature.qualifiers.get("product",[""])[0]
                gene=feature.qualifiers.get("gene",[""])[0]
                protein_id=feature.qualifiers["protein_id"][0]
                GO_process=feature.qualifiers.get("GO_process",[""])[0]
                GO_function=feature.qualifiers.get("GO_function",[""])[0]
                GO_component=feature.qualifiers.get("GO_component",[""])[0]
                lista.append([locus_tag,product,gene,protein_id,GO_process,GO_function,GO_component])
data=pd.DataFrame(lista,columns=["locus","product","gene","protein_id","GO_process","GO_function","GO_component"])
data
                



,locus,product,gene,protein_id,GO_process,GO_function,GO_component
0,LAU37_RS00005,DUF1257 domain-containing protein,,WP_275983370.1,,,
1,LAU37_RS00010,hypothetical protein,,WP_250123595.1,,,
2,LAU37_RS00015,anthranilate synthase,,WP_250123596.1,GO:0000162 - tryptophan biosynthetic process [...,GO:0004049 - anthranilate synthase activity [E...,
3,LAU37_RS00020,indole-3-glycerol phosphate synthase TrpC,trpC,WP_250123597.1,GO:0006568 - tryptophan metabolic process [Evi...,GO:0004425 - indole-3-glycerol-phosphate synth...,
4,LAU37_RS00025,tryptophan synthase subunit alpha,trpA,WP_250123598.1,GO:0000162 - tryptophan biosynthetic process [...,GO:0004834 - tryptophan synthase activity [Evi...,
...,...,...,...,...,...,...,...
6005,LAU37_RS31385,IS4 family transposase,,WP_256479003.1,,GO:0004803 - transposase activity [Evidence IEA],
6006,LAU37_RS31390,ATP-binding protein,,WP_250126520.1,,,
6007,LAU37_RS31400,hypothetical protein,,WP_250126521.1,,,
6008,LAU37_RS31405,hypothetical protein,,WP_250126522.1,,,


In [123]:
import re
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta"
motivi=[".AGT.{4,11}ACT.",".TAGT.{5,10}(?<!A)CT."]
from Bio import SeqIO
#parse the fasta file
diz={}
j=0
for record in SeqIO.parse(file, "fasta"):
    seq=str(record.seq)
    #use the finditer method to find all the matches of the regular expression
    for m in motivi:
        for match in re.finditer(m, str(seq)):
            x=match.group()
            s,e=match.start(),match.end()
            # print(x,s,e,seq[s],seq[e-1])
            x=x[0].lower()+x[1:-1]+x[-1].lower()
            while True:
                if s==0 or e==len(seq):
                    break
                if seq[s]==rev(seq[e-1]):
                    x=seq[s].lower()+x+seq[e-1].lower()
                    s=s-1
                    e=e+1
                elif seq[s]!=rev(seq[e-1]):
                    break

            #store the match in a dictionary
            if record.id in diz:
                diz[record.id].append(x)
            else:
                diz[record.id]=[x]
import pandas as pd
df=pd.Series(diz,name="motivi")

    

In [92]:
df=pd.merge(df,data,on="locus",how="left")
df_path.index.name="protein_id"
df_br.index.name="protein_id"
df=pd.merge(df,df_path,on="protein_id",how="left")
#merge df and df_br on protein_id
df=pd.merge(df,df_br,on="protein_id",how="left")
df

,locus,motivi,product,gene,protein_id,GO_process,GO_function,GO_component,pathways,Kegg Brite
0,LAU37_RS00020,[tTAGTTACCCCTg],indole-3-glycerol phosphate synthase TrpC,trpC,WP_250123597.1,GO:0006568 - tryptophan metabolic process [Evi...,GO:0004425 - indole-3-glycerol-phosphate synth...,,"Metabolic pathways, Biosynthesis of secondary ...",ko01000 - Enzymes
1,LAU37_RS00055,[gAGTTCAATACTa],HpsJ family protein,,WP_250123604.1,,,,NaN,NaN
2,LAU37_RS00060,[tAGTTTTGCTTTACTg],hypothetical protein,,WP_250123605.1,,,,NaN,ko02000 - Transporters
3,LAU37_RS00110,[aTAGTAAGGAGTACCTa],Ycf66 family protein,,WP_250123614.1,,,,NaN,NaN
4,LAU37_RS00135,[aAGTACCCATACTc],hypothetical protein,,WP_250123619.1,,,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2396,LAU37_RS31285,[gAGTATCAACTa],GTP cyclohydrolase I FolE,folE,WP_250126505.1,GO:0042559 - pteridine-containing compound bio...,GO:0003934 - GTP cyclohydrolase I activity [Ev...,,"Metabolic pathways, Biosynthesis of cofactors,...",ko01000 - Enzymes
2397,LAU37_RS31330,"[cAGTTTTTATCCACTa, cTAGTGAACTTTTAGCTa]",hypothetical protein,,WP_250126513.1,,,,NaN,NaN
2398,LAU37_RS31345,"[tAGTTTATAGAGACTt, aTAGTTGACAAGCGCTc]",IS4 family transposase,,WP_250126516.1,,GO:0004803 - transposase activity [Evidence IEA],,NaN,NaN
2399,LAU37_RS31360,[ccTAGTCGCTTTCCCTgg],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
#order df by number of motifs
df["n_motivi"]=df["motivi"].apply(lambda x: len(x))
df.sort_values(by="n_motivi",ascending=False)


,locus,motivi,product,gene,protein_id,GO_process,GO_function,GO_component,pathways,Kegg Brite,n_motivi
1201,LAU37_RS04245,"[tAGTGGTTACTc, aaAGTGTTCTTACTAAACTtt, tAGTTCCT...",2Fe-2S iron-sulfur cluster-binding protein,,WP_250124385.1,,GO:0009055 - electron transfer activity [Evide...,,NaN,NaN,7
1360,LAU37_RS07840,"[cttttAGTAATTAGCCTCACTaaaag, tAGTCTCCATACTt, t...",glycosyltransferase,,WP_250125026.1,,,,NaN,NaN,6
937,LAU37_RS25215,"[ggAGTAGCCAAACTcc, ttAGTTTGTACTaa, aAGTAGTTAGC...",cation:proton antiporter,,WP_250123190.1,GO:0006812 - monoatomic cation transport [Evid...,,GO:0016020 - membrane [Evidence IEA],NaN,NaN,6
2146,LAU37_RS28725,"[ttAGTTGCTAACCCAACTaa, ttAGTATAAACTGAGACTaa, a...",type II toxin-antitoxin system RelE/ParE famil...,,WP_250126688.1,,,,NaN,NaN,5
612,LAU37_RS17070,"[aaAGTCTGTCTCACTtt, aaAGTCAGACCGCAAAACTtt, aaA...",MBL fold metallo-hydrolase,,WP_250121690.1,,,,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...
988,LAU37_RS26555,[gAGTCAGATTACTa],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
983,LAU37_RS26400,[aTAGTAGTAGGAGCGCTc],hypothetical protein,,WP_250123409.1,,,,NaN,NaN,1
979,LAU37_RS26255,[aTAGTCTCTCGCTa],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
978,LAU37_RS26190,[gAGTAGTTACTa],hypothetical protein,,WP_250123369.1,,,,NaN,NaN,1


In [71]:
#save df to excel
df.to_excel("/home/davide/Desktop/tabellamotiviCCMEE29.xlsx")

In [94]:
#find all duplicates in df of protein_id
df[df.duplicated(subset="protein_id",keep=False)]
#drop rows with protein_id null
df.dropna(subset=["protein_id"],inplace=True)
df

,locus,motivi,product,gene,protein_id,GO_process,GO_function,GO_component,pathways,Kegg Brite,n_motivi
0,LAU37_RS00020,[tTAGTTACCCCTg],indole-3-glycerol phosphate synthase TrpC,trpC,WP_250123597.1,GO:0006568 - tryptophan metabolic process [Evi...,GO:0004425 - indole-3-glycerol-phosphate synth...,,"Metabolic pathways, Biosynthesis of secondary ...",ko01000 - Enzymes,1
1,LAU37_RS00055,[gAGTTCAATACTa],HpsJ family protein,,WP_250123604.1,,,,NaN,NaN,1
2,LAU37_RS00060,[tAGTTTTGCTTTACTg],hypothetical protein,,WP_250123605.1,,,,NaN,ko02000 - Transporters,1
3,LAU37_RS00110,[aTAGTAAGGAGTACCTa],Ycf66 family protein,,WP_250123614.1,,,,NaN,NaN,1
4,LAU37_RS00135,[aAGTACCCATACTc],hypothetical protein,,WP_250123619.1,,,,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2395,LAU37_RS31230,[ttAGTAAAAGCGTAAAACTaa],alpha/beta hydrolase,,WP_250126497.1,,,,NaN,NaN,1
2396,LAU37_RS31285,[gAGTATCAACTa],GTP cyclohydrolase I FolE,folE,WP_250126505.1,GO:0042559 - pteridine-containing compound bio...,GO:0003934 - GTP cyclohydrolase I activity [Ev...,,"Metabolic pathways, Biosynthesis of cofactors,...",ko01000 - Enzymes,1
2397,LAU37_RS31330,"[cAGTTTTTATCCACTa, cTAGTGAACTTTTAGCTa]",hypothetical protein,,WP_250126513.1,,,,NaN,NaN,2
2398,LAU37_RS31345,"[tAGTTTATAGAGACTt, aTAGTTGACAAGCGCTc]",IS4 family transposase,,WP_250126516.1,,GO:0004803 - transposase activity [Evidence IEA],,NaN,NaN,2


In [95]:
#save df to excel

df=df.sort_values(by="n_motivi",ascending=False)
df_tabella=df

In [79]:
df

,locus,motivi,product,gene,protein_id,GO_process,GO_function,GO_component,pathways,Kegg Brite,n_motivi
1201,LAU37_RS04245,"[tAGTGGTTACTc, aaAGTGTTCTTACTAAACTtt, tAGTTCCT...",2Fe-2S iron-sulfur cluster-binding protein,,WP_250124385.1,,GO:0009055 - electron transfer activity [Evide...,,NaN,NaN,7
1360,LAU37_RS07840,"[cttttAGTAATTAGCCTCACTaaaag, tAGTCTCCATACTt, t...",glycosyltransferase,,WP_250125026.1,,,,NaN,NaN,6
937,LAU37_RS25215,"[ggAGTAGCCAAACTcc, ttAGTTTGTACTaa, aAGTAGTTAGC...",cation:proton antiporter,,WP_250123190.1,GO:0006812 - monoatomic cation transport [Evid...,,GO:0016020 - membrane [Evidence IEA],NaN,NaN,6
2194,LAU37_RS29710,"[aAGTTGACAACTa, gAGTGGGTCAAAATAACTt, taaaAGTAG...","ribbon-helix-helix protein, CopG family",,WP_250126268.1,GO:0006355 - regulation of DNA-templated trans...,,,NaN,NaN,5
2280,LAU37_RS28395,"[aAGTGCAGGTAGCACTc, tAGTGTTCACTc, aaAGTTTATTAG...",IS5 family transposase,,WP_250124042.1,,GO:0004803 - transposase activity [Evidence IEA],,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...
956,LAU37_RS25730,[gAGTTTGAAAACTg],methionine gamma-lyase family protein,,WP_250123281.1,,,,"Metabolic pathways, Biosynthesis of secondary ...","ko01000 - Enzymes, ko04147 - Exosome",1
955,LAU37_RS25715,[gAGTCAAGCACTa],hypothetical protein,,WP_250123278.1,,,,NaN,NaN,1
953,LAU37_RS25670,[aaAGTATGAATAACTtt],DUF2993 domain-containing protein,,WP_250123271.1,,,,NaN,NaN,1
952,LAU37_RS25660,[aaaAGTTTTGAGACTttt],alpha/beta hydrolase,,WP_250123269.1,,,,NaN,NaN,1


In [99]:
import pandas as pd
file="/home/davide/Downloads/geniregolatiLexAincianobatteri.xlsx"
df_evi=pd.read_excel(file)
df_evi["old_locus_tag"]=df_evi["old_locus_tag"].str.split("-")
df_evi=df_evi.explode("old_locus_tag")
df_evi["old_locus_tag"]=df_evi["old_locus_tag"].str.split()
df_evi=df_evi.explode("old_locus_tag")
df_evi

,old_locus_tag,gene_omologo,motivo_omologo,Organismo/evidenze,Pubblicazione
0,all0875,glgB,AGTataaaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
1,all2563,talB,AGTcttaacaaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
2,all4713,galE,AGTagttgtcaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
3,all4121,petH,AGTTagaaggtgAACa,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
4,alr4641,prxA,ACAGaTcgtaatcaAACTGT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
...,...,...,...,...,...
181,all4148,"petF, fd1",AGTatcagaattgaACT\nAGTtatatACT\nAGTataaaaaaaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
182,all1430,fdxH,AGTgtccaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
183,all2919,fd2,AGTttgggtgaaaACT\nAGTttcacccaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
184,alr0784,fd3,AGTagttgACT\nAGTtacttcACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"


In [100]:
file="/home/davide/Desktop/genomiChro/blastvsPCC7120/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375.1VsNostocPCC7120.txt"
df=pd.read_csv(file,sep="\t",header=None)
#set the columns name
df.columns=["protein_id","subject","identity","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
#drop the columns that are not useful
df.drop(["mismatch","gapopen","qstart","qend","sstart","send","bitscore"],axis=1,inplace=True)
#create a new column with the old_locus_tag
df.loc[:,"old_locus_tag"]=df.loc[:,"subject"].str.extract(r"\((.*?)\)",expand=False)
#keep only the first hit for each query
#df.drop_duplicates(subset="query",keep="first",inplace=True)
#order the dataframe by evalue
df.sort_values(by="evalue",inplace=True)
#drop duplicates of old_locus_tag keeping the first with the lowest evalue
df.drop_duplicates(subset="old_locus_tag",keep="first",inplace=True)
#create a new column with the information of the blastp
df["info_blastp"]='id:'+df["identity"].astype(str)+"%, len:"+df["length"].astype(str)+", evalue:"+df["evalue"].astype(str)
#drop the columns that are not useful
df.drop(columns=["subject","identity","length","evalue"],inplace=True)
#save the dataframe in a new file
#join with df_evi
df=pd.merge(df,df_evi,how="inner",on="old_locus_tag")
df

,protein_id,old_locus_tag,info_blastp,gene_omologo,motivo_omologo,Organismo/evidenze,Pubblicazione
0,WP_250122389.1,alr1742,"id:87.874%, len:635, evalue:0.0",dnaK,AGTAttaaTtCT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
1,WP_250122400.1,alr3343,"id:80.769%, len:416, evalue:0.0",fabF,AAGTcaagAtTT,Anabaena PCC 7120: Bioinformatica,"Kumar et al., 2018"
2,WP_250122454.1,alr3199,"id:72.886%, len:343, evalue:0.0",DNase,AGTtaagaatccgACT,Anabaena PCC 7120: Bioinformatica,"Kumar et al., 2018"
3,WP_250122461.1,alr3716,"id:79.441%, len:1002, evalue:0.0",uvrA,AGTattattaaaaACT,Anabaena PCC 7120: Bioinformatica,"Kumar et al., 2018"
4,WP_250122313.1,alr4853,"id:79.897%, len:388, evalue:0.0",NaN,tGTGgcgggaaCACT,Anabaena PCC 7120: sovraespressione+RT-qPCR +EMSA,"Kumar et al, 2023"
...,...,...,...,...,...,...,...
173,WP_250122117.1,all3850,"id:40.909%, len:22, evalue:0.062",psaI,AGTctgggcatcACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
174,WP_250123945.1,all0688,"id:32.895%, len:76, evalue:0.067",hupS,AGTggcaaaacaACT,Anabaena PCC 7120: Bioinformatica,"Kumar et al., 2018"
175,WP_250126523.1,all4688,"id:41.935%, len:31, evalue:0.52",NaN,AGTAccgtTtCT,Anabaena PCC 7120: sovraespressione+RT-qPCR +EMSA,"Kumar et al, 2023"
176,WP_250126276.1,all2110,"id:41.667%, len:24, evalue:0.85",NaN,AGTTCtctGAACa,Anabaena PCC 7120: sovraespressione+RT-qPCR +EMSA,"Kumar et al, 2023"


In [105]:
df2=pd.merge(df_tabella,df,on="protein_id",how="left")
df2

,locus,motivi,product,gene,protein_id,GO_process,GO_function,GO_component,pathways,Kegg Brite,n_motivi,old_locus_tag,info_blastp,gene_omologo,motivo_omologo,Organismo/evidenze,Pubblicazione
0,LAU37_RS04245,"[tAGTGGTTACTc, aaAGTGTTCTTACTAAACTtt, tAGTTCCT...",2Fe-2S iron-sulfur cluster-binding protein,,WP_250124385.1,,GO:0009055 - electron transfer activity [Evide...,,NaN,NaN,7,asl0884,"id:77.551%, len:98, evalue:5.7899999999999994e-55",psbM,tGTAatTACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
1,LAU37_RS07840,"[cttttAGTAATTAGCCTCACTaaaag, tAGTCTCCATACTt, t...",glycosyltransferase,,WP_250125026.1,,,,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
2,LAU37_RS25215,"[ggAGTAGCCAAACTcc, ttAGTTTGTACTaa, aAGTAGTTAGC...",cation:proton antiporter,,WP_250123190.1,GO:0006812 - monoatomic cation transport [Evid...,,GO:0016020 - membrane [Evidence IEA],NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
3,LAU37_RS29710,"[aAGTTGACAACTa, gAGTGGGTCAAAATAACTt, taaaAGTAG...","ribbon-helix-helix protein, CopG family",,WP_250126268.1,GO:0006355 - regulation of DNA-templated trans...,,,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,LAU37_RS28395,"[aAGTGCAGGTAGCACTc, tAGTGTTCACTc, aaAGTTTATTAG...",IS5 family transposase,,WP_250124042.1,,GO:0004803 - transposase activity [Evidence IEA],,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279,LAU37_RS25730,[gAGTTTGAAAACTg],methionine gamma-lyase family protein,,WP_250123281.1,,,,"Metabolic pathways, Biosynthesis of secondary ...","ko01000 - Enzymes, ko04147 - Exosome",1,NaN,NaN,NaN,NaN,NaN,NaN
2280,LAU37_RS25715,[gAGTCAAGCACTa],hypothetical protein,,WP_250123278.1,,,,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2281,LAU37_RS25670,[aaAGTATGAATAACTtt],DUF2993 domain-containing protein,,WP_250123271.1,,,,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2282,LAU37_RS25660,[aaaAGTTTTGAGACTttt],alpha/beta hydrolase,,WP_250123269.1,,,,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
df2.to_excel("/home/davide/Desktop/tabellamotiviCCMEE29conblastp.xlsx")

In [107]:
#remove duplicates from df2 of locus
df2.drop_duplicates(subset="locus",keep="first",inplace=True)
df2.to_excel("/home/davide/Desktop/tabellamotiviCCMEE29nodupl.xlsx")

In [108]:
#save df_br and df_path to csv
df_br.to_csv("/home/davide/Desktop/kegg_brite.csv")
df_path.to_csv("/home/davide/Desktop/kegg_pathways.csv")